<a href="https://colab.research.google.com/github/AbrerAsif07/AbrerAsif07/blob/main/ReAct_Agent_Lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define the calculator tool

from sympy import sympify
def calculator(expression):
    try:
        result = sympify(expression)
        return result
    except Exception as e:
        return f"Error in calculation: {e}"

In [ ]:
calculator("1*2*3*4*5")

120

In [ ]:
import re

def extract_action_and_input(text):
  action = re.search(r"Action: (.*)", text)
  action_input = re.search(r"Action Input: (.*)", text)
  return action.group(1).strip() if action else None, action_input.group(1).strip() if action else None

extract_action_and_input("""
Thought: To calculate the square root of 144, I can use the math library in Python or a calculator.

Action: Calculator
Action Input: sqrt(144)
""")

('Calculator', 'sqrt(144)')

In [ ]:
from google.colab import userdata

from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('OPENROUTER_API_KEY'),
)

user_prompt = "Calculate the square root of 144"

chat_history = [
    {
      "role": "system",
      "content": """
      You have access to the following tools:
      Calculator: Use this when you want to do math. Use SymPy expressions, eg: 2 + 2

      Use the following format:

      Question: the input question you must answer
      Thought: you should always think about what to do
      Action: the action to take, should be one of [Calculator]
      Action Input: the input to the action
      Observation: the result of teh action
      ... (the Thought/Action/Observation can repeat any number of times)
      Thought: I now know the final answer!
      Final Answer: the answer to the original input question
      """
    },
    {
      "role": "user",
      "content": f"Question: {user_prompt}"
    }
  ]

import re

while True:
  completion = client.chat.completions.create(
    model="meta-llama/llama-3.2-90b-vision-instruct:free",
    messages=chat_history,
    stop=["Observation:"]
  )
  response_text = completion.choices[0].message.content
  action, action_input = extract_action_and_input(response_text)
  print(response_text)
  # We want to see if the LLM took an action
  if action == "Calculator":
    action_result = calculator(action_input)
    print(f"Observation: {action_result}")
    chat_history.extend([
      { "role": "assistant", "content": response_text },
      { "role": "user", "content": f"Observation: {action_result}" }
    ])
  else:
    break

Thought: To find the square root of 144, I'll use the calculator.

Action: Calculator 
Action Input: SymPy.sqrt(144)

Observation: Error in calculation: 'Symbol' object has no attribute 'sqrt'
Thought: It seems that SymPy isn't available for this action, I'll use a different method.

Action: Calculator 
Action Input: 144 ** 0.5
Observation: 12.0000000000000
Thought: I now know the final answer!

Final Answer: The square root of 144 is 12.


In [ ]:
print(response_text)

Thought: To calculate the square root of 144, I can use the math library in Python or a calculator.

Action: Calculator 
Action Input: sqrt(144) 




In [ ]:
calculator("sqrt(144)")

12

In [ ]:
chat_history

[{'role': 'system',
  'content': '\n      You have access to the following tools:\n      Knife\n      Machete\n      Chainsaw\n      Rubber band\n\n      Use the following format:\n\n      Question: the input question you must answer\n      Thought: you should always think about what to do\n      Action: the action to take, should be one of [Knife, Machete, Chainsaw, Rubber band]\n      '},
 {'role': 'user',
  'content': '\n      Question: I want to open a present.\n      '}]

In [ ]:
chat_history.extend([
    { "role": "assistant", "content": response_text },
    { "role": "user", "content": "Observation: The knife did not work! A tree has fallen down between you and the present." }
])

In [ ]:
chat_history

[{'role': 'system',
  'content': '\n      You have access to the following tools:\n      Knife\n      Machete\n      Chainsaw\n      Rubber band\n\n      Use the following format:\n\n      Question: the input question you must answer\n      Thought: you should always think about what to do\n      Action: the action to take, should be one of [Knife, Machete, Chainsaw, Rubber band]\n      '},
 {'role': 'user',
  'content': '\n      Question: I want to open a present.\n      '},
 {'role': 'assistant',
  'content': 'Thought: I need to carefully remove the wrapping without damaging the gift. The wrapping is likely held in place with tape or by being wrapped tightly around the gift.\n\nAction: Rubber band (or if not using a elastic wrapping, then Knife).'},
 {'role': 'user',
  'content': 'Observation: The knife did not work! A tree has fallen down between you and the present.'}]

In [ ]:
completion = client.chat.completions.create(
  model="meta-llama/llama-3.2-90b-vision-instruct:free",
  messages=chat_history
)
response_text = completion.choices[0].message.content

In [ ]:
response_text

'Action: Chainsaw'